In [ ]:
# Copyright 2025 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

### Event 처리 분석

In [5]:
import os
from dotenv import load_dotenv
import argparse

from google.genai import types
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

from agent import root_agent

In [ ]:
# 환경 변수 로드.

from dotenv import dotenv_values
config = dotenv_values("../../.env")
for key, value in config.items():
    print(f"{key}: {value}")

GOOGLE_GENAI_USE_VERTEXAI: TRUE
GOOGLE_CLOUD_PROJECT: ai-hangsik
GOOGLE_CLOUD_LOCATION: us-central1
GOOGLE_GENAI_MODEL: gemini-2.5-flash
GOOGLE_GENAI_LIVE_MODEL: gemini-live-2.5-flash
AGENT_ENGINE_BUCKET: gs://agent_engine_0120
AGENTOPS_API_KEY: 43150e85-1c93-4c65-8526-3ac50cf1bcfc


In [12]:
APP_NAME = "AI_assistant"
USER_ID = "Forusone"

# 세션 서비스를 생성하고 초기화합니다.
# 세션 서비스는 메모리에서 세션을 관리합니다.
# InMemorySessionService는 ADK에서 제공하는 기본 세션 서비스입니다.
session_service = InMemorySessionService()
session = await session_service.create_session(app_name=APP_NAME,
                                        user_id=USER_ID,
                                        state={"initial_key": "initial_value"})
# 에이전트 러너를 생성합니다.
# 러너는 에이전트를 세션 서비스와 연결하여 사용자 입력을 처리합니다.
runner = Runner(agent=root_agent,
                app_name=session.app_name,
                session_service=session_service)

In [ ]:
print(f"AI Agent를 종료하려면 'exit' 또는 'quit'을 입력하세요.")

while True:
    print("\n👤 User :   ")

    # 사용자 입력에서 쿼리 가져오기
    query = input("")
    if query.strip().lower() in ["exit", "quit"]:
        break
    else:
        print(query)

        # 사용자 입력으로부터 Content 객체를 생성합니다.
        content = types.Content(role='user', parts=[types.Part(text=query)])

        # 러너를 사용하여 에이전트 이벤트를 비동기적으로 실행합니다.
        events = runner.run_async(user_id=session.user_id,
                                session_id=session.id,
                                new_message=content,
                                )
        # 이벤트 스트림을 비동기적으로 처리합니다.
        async for event in events:
            print("\n\n-------------------------")
            print(f"event.invocation_id: {event.invocation_id}")
            print(f"event.author: {event.author}")
            print(f"event.actions: {event.actions}")
            print(f"event.branch: {event.branch}")    
            print(f"event.id: {event.id}")
            print(f"event.is_final_response(): {event.is_final_response()}")        
            
            # 이벤트에 근거(grounding) 콘텐츠(참조 데이터)가 포함되어 있으면 출력합니다.
            if event.grounding_metadata is not None:
                print("\n\n-----------< Grounding service information >--------------")

                if event.grounding_metadata.grounding_chunks is not None:
                    for grounding_chunk in event.grounding_metadata.grounding_chunks:
                        print(f"\n\n--------[ Title: {grounding_chunk.web.title} ]----------")
                        print(f"* grounding_chunk.web.domain: {grounding_chunk.web.domain}")
                        print(f"* grounding_chunk.web.url: {grounding_chunk.web.uri}")
                    
            if event.is_final_response():
                final_response = event.content.parts[0].text            
                print(f"\n 🤖 AI Assistant: {final_response}\n")


AI Agent를 종료하려면 'exit' 또는 'quit'을 입력하세요.

👤 User :   
생성형 AI 란 ?


-------------------------
event.invocation_id: e-698c268d-e390-4218-a264-ecc425430c1a
event.author: search_agent
event.actions: skip_summarization=None state_delta={} artifact_delta={} transfer_to_agent=None escalate=None requested_auth_configs={} requested_tool_confirmations={} compaction=None end_of_agent=None agent_state=None rewind_before_invocation_id=None
event.branch: None
event.id: b21e42a1-0f8f-40fc-b153-48f4fc04da66
event.is_final_response(): True


-----------< Grounding service information >--------------


--------[ Title: amazon.com ]----------
* grounding_chunk.web.domain: amazon.com
* grounding_chunk.web.url: https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGLqO11PcR2nPR_ytjUOCfgaox2zDuO5ky4XEN8-dy3a5rGxBOv_zbOXP0bJ_PYsTER8Q3S6-AurG8bCkXYMhOewMmFsorMCUXYIYcm3qpkRMvq86UWWUSlNT6xHPIdd8Gl8cmy_qIOQd1fCQ==


--------[ Title: ibm.com ]----------
* grounding_chunk.web.domain: ibm.com
* groun